In [6]:
!pip install --upgrade pip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.1 MB 29.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3


### Importing Haystack library
Haystack is a library which helps to finetune the model easily.

In [7]:
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-kxms5seg/farm-haystack_e0892166bf464eb582a7349e1f692882
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-kxms5seg/farm-haystack_e0892166bf464eb582a7349e1f692882
  Resolved https://github.com/deepset-ai/haystack.git to commit 6ce2d296f4cd34059905817f0a2717801ba27c61
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached transformers-4.21.2-py3-none-any.whl (4.7 MB)
  Using cached mlflow-1.30.0-py3-none-any.whl (17.0 MB)
  Using cached langdetect-1.0.9.tar.gz (981 kB)
  Preparing metadata (setup.py) ... done
  Using cached python-docx-0.8.11.tar.gz (5.6 MB)
  Preparing metadata (setup.py) ... done
  Using cached tika-1

### Importing FARMReader class

In [9]:
from haystack.nodes import FARMReader

## Here, I'm using the pretrained model called RoBERTa trained in squad dataset for question answering. 

In [61]:
model_name = "deepset/roberta-base-squad2" 

### Downloading the model parameters.

In [13]:
from haystack.nodes import FARMReader
reader = FARMReader(model_name_or_path=model_name, use_gpu=True)
data_dir = "data/"  # path of the training data

### Traing the model with the dataset.

In [16]:
reader.train(data_dir=data_dir, train_filename="dev-v1.1.json", use_gpu=True, n_epochs=3, save_dir="my_model") # save_dir is used for saving the model.

Train epoch 2/2 (Cur. train loss: 0.1667): 100%|██████████| 1203/1203 [10:22<00:00,  1.93it/s]


### Using the saved model

In [17]:
model = FARMReader(model_name_or_path="my_model")

### Evaluating the model on the train data.

In [30]:
reader_eval_results = model.eval_on_file("data/", "dev-v2.0.json", device="cuda")

- instead of giving you full control over which labels to use, this method always returns three types of metrics: combined (no suffix), text_answer ('_text_answer' suffix) and no_answer ('_no_answer' suffix) metrics.
- instead of comparing predictions with labels on a string level, this method compares them on a token-ID level. This makes it unable to do any string normalization (e.g. normalize whitespaces) beforehand.
Hence, results might slightly differ from those of `Pipeline.eval()`
.If you are just about starting to evaluate your model consider using `Pipeline.eval()` instead.
Evaluating: 100%|██████████| 274/274 [03:38<00:00,  1.25it/s]


### Here, we can see the Accuracy of the model. We got almost 99.8 accuracy here.  

In [31]:
reader_eval_results

{'EM': 50.32201914708442,
 'f1': 51.39102405192665,
 'top_n_accuracy': 99.72149695387293,
 'top_n': 4,
 'EM_text_answer': 90.62393453801569,
 'f1_text_answer': 92.7178428838454,
 'top_n_accuracy_text_answer': 99.5908625980225,
 'top_n_EM_text_answer': 96.99965905216501,
 'top_n_f1_text_answer': 98.90805374537153,
 'Total_text_answer': 2933,
 'EM_no_answer': 8.285917496443812,
 'f1_no_answer': 8.285917496443812,
 'top_n_accuracy_no_answer': 99.85775248933145,
 'Total_no_answer': 2812}

### Initializing some context for Question Answering

In [33]:
context = """No Country for Old Men is a 2007 American neo-Western crime thriller film written and directed by Joel and Ethan Coen, based on Cormac McCarthy's 2005 novel of the same name.
Starring Tommy Lee Jones, Javier Bardem, and Josh Brolin, the film is set in the desert landscape of 1980 West Texas.
The film revisits the themes of fate, conscience, and circumstance that the Coen brothers had explored in the films Blood Simple (1984), Raising Arizona (1987), and Fargo (1996).
The film follows three main characters: Llewelyn Moss (Brolin), a Vietnam War veteran and welder who stumbles upon a large sum of money in the desert; 
Anton Chigurh (Bardem), a hitman who is tasked with recovering the money; and Ed Tom Bell (Jones), a local sheriff investigating the crime. 
The film also stars Kelly Macdonald as Moss's wife Carla Jean, and Woody Harrelson as a bounty hunter seeking Moss and the return of the $2 million.
No Country for Old Men premiered in competition at the 2007 Cannes Film Festival on May 19.
The film became a commercial success, grossing $171 million worldwide against the budget of $25 million. 
Critics praised the Coens' direction and screenplay and Bardem's performance, and the film won 76 awards from 109 nominations from multiple organizations; 
it won four awards at the 80th Academy Awards (including Best Picture), three British Academy Film Awards (BAFTAs), and two Golden Globes.
The American Film Institute listed it as an AFI Movie of the Year,[6] and the National Board of Review selected it as the best of 2007"""

### Initializing Question

In [34]:
ques = 'In which year did No Country for Old Men released?'

### Here, we can see the answer of the question that the model is predicting. 
### We can see the answer with the score of it. Also we can see other possible answers with the scores. 

In [43]:
ans = model.predict_on_texts(ques,[context])
ans

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 11.89 Batches/s]


{'query': 'In which year did No Country for Old Men released?',
 'no_ans_gap': 17.03927230834961,
 'answers': [<Answer {'answer': '2007', 'type': 'extractive', 'score': 0.9891924858093262, 'context': "No Country for Old Men is a 2007 American neo-Western crime thriller film written and directed by Joel and Ethan Coen, based on Cormac McCarthy's 2005", 'offsets_in_document': [{'start': 28, 'end': 32}], 'offsets_in_context': [{'start': 28, 'end': 32}], 'document_id': 'ea91e9aaa97e3f4798d9fe101fc4d4fd', 'meta': {}}>,
  <Answer {'answer': '2007', 'type': 'extractive', 'score': 0.012154441326856613, 'context': 'f the $2 million.\nNo Country for Old Men premiered in competition at the 2007 Cannes Film Festival on May 19.\nThe film became a commercial success, gr', 'offsets_in_document': [{'start': 969, 'end': 973}], 'offsets_in_context': [{'start': 73, 'end': 77}], 'document_id': 'ea91e9aaa97e3f4798d9fe101fc4d4fd', 'meta': {}}>]}

In [44]:
ans['answers']

[<Answer {'answer': '2007', 'type': 'extractive', 'score': 0.9891924858093262, 'context': "No Country for Old Men is a 2007 American neo-Western crime thriller film written and directed by Joel and Ethan Coen, based on Cormac McCarthy's 2005", 'offsets_in_document': [{'start': 28, 'end': 32}], 'offsets_in_context': [{'start': 28, 'end': 32}], 'document_id': 'ea91e9aaa97e3f4798d9fe101fc4d4fd', 'meta': {}}>,
 <Answer {'answer': '2007', 'type': 'extractive', 'score': 0.012154441326856613, 'context': 'f the $2 million.\nNo Country for Old Men premiered in competition at the 2007 Cannes Film Festival on May 19.\nThe film became a commercial success, gr', 'offsets_in_document': [{'start': 969, 'end': 973}], 'offsets_in_context': [{'start': 73, 'end': 77}], 'document_id': 'ea91e9aaa97e3f4798d9fe101fc4d4fd', 'meta': {}}>]

In [50]:
ques = 'what is the budget of the film?'

In [59]:
ques = 'who is the director of No Country for Old Men'

### Here, I'm defining a pipe line for the Question Answering.

In [62]:
from haystack import Pipeline, Document
from haystack.utils import print_answers

p = Pipeline()
p.add_node(component=model, name="Reader", inputs=["Query"])
res = p.run(
    query=ques, documents=[Document(content=context)]
)
print_answers(res,details="medium")

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 19.49 Batches/s]


Query: who is the director of No Country for Old Men
Answers:
[   {   'answer': 'Joel and Ethan Coen',
        'context': ' American neo-Western crime thriller film written and '
                   'directed by Joel and Ethan Coen, based on Cormac '
                   "McCarthy's 2005 novel of the same name.\n"
                   'Starrin',
        'score': 0.9967261552810669},
    {   'answer': 'Coens',
        'context': 'illion worldwide against the budget of $25 million. \n'
                   "Critics praised the Coens' direction and screenplay and "
                   "Bardem's performance, and the film won 76",
        'score': 0.10386646538972855}]
